In [ ]:
# importing necessary libraries
import os
from IPython.display import clear_output
from packages import esp4_import_functions as esp4fns
from numpy import zeros, shape

In [ ]:
# path to the BBX files
# This path should be updated to the actual location of your BBX files.
path = "/Users/santapile/Santas/Samples/StripeEasy/200924_BESSY_Oct20_45deg/STXM/Maxymus_Oct20/Santa/rawTest/201030/"

In [ ]:
# Converting BBX files to CSV format
# The choice parameter can be "magnetic" or "chemical" depending on which contrast you want to extract.
# Here, we are using "magnetic" as an example.
esp4fns.BBX2CSV(path, choice = "magnetic")

7 / 7
magnetic
Done with /Users/santapile/Santas/Samples/StripeEasy/200924_BESSY_Oct20_45deg/STXM/Maxymus_Oct20/Santa/rawTest/201030/
